In [1]:
import torch
from transformers import BertForNextSentencePrediction
from transformers import AutoTokenizer

In [2]:
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\amps\anaconda3\envs\dl_study\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amps\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

In [4]:
# 정수 인코딩
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

In [5]:
print(encoding['input_ids'])
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)  # 101 - [CLS]
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)  # 102 - [SEP]

tensor([[  101,  1999,  3304,  1010, 10733,  2366,  1999,  5337, 10906,  1010,
          2107,  2004,  2012,  1037,  4825,  1010,  2003,  3591,  4895, 14540,
          6610,  2094,  1012,   102, 10733,  2003,  8828,  2007,  1996,  2224,
          1997,  1037,  5442,  1998,  9292,  1012,  1999, 10017, 10906,  1010,
          2174,  1010,  2009,  2003,  3013,  2046, 17632,  2015,  2000,  2022,
          8828,  2096,  2218,  1999,  1996,  2192,  1012,   102]])
[CLS] : 101
[SEP] : 102


In [6]:
# 인코딩 결과 디코딩
# BERT에서 두 개의 문장이 입력으로 들어갈 경우에는 맨 앞에는 [CLS] 토큰이 존재하고, 
# 첫번째 문장이 끝나면 [SEP] 토큰, 그리고 두번째 문장이 종료되었을 때 다시 추가적으로 [SEP] 토큰이 추가됩니다.
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


In [7]:
# 세그먼트 인코딩 결과를 확인
# 0이 연속적으로 등장하다가 어느 순간부터 1이 연속적으로 등장하는데, 
# 이는 [CLS] 토큰의 위치부터 첫번째 문장이 끝나고나서 등장한 [SEP] 토큰까지의 위치에는 0이 등장하고, 다음 두번째 문장부터는 1이 등장하는 것입니다. 
# token_type_ids에서는 0과 1로 두 개의 문장을 구분하고 있습니다.
print(encoding['token_type_ids'])

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


# 다음 문장 예측

In [8]:
#  모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값을 리턴
pred = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])
probs = torch.nn.functional.softmax(pred.logits, dim=1)  # Softmax 적용하여 확률 얻기
print(probs)  # 모델이 예측한 레이블은 0

tensor([[1.0000e+00, 2.8382e-06]], grad_fn=<SoftmaxBackward0>)


In [10]:
# BERT는 사전 학습 당시 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습
# 위 데이터는 이어진 문장으로 분류
next_sentence_label = torch.argmax(probs, dim=1).item()  # 예측된 라벨 얻기
print('최종 예측 레이블 :', next_sentence_label)

최종 예측 레이블 : 0


In [11]:
# 상관없는 두 개의 문장
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

pred = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])
probs = torch.nn.functional.softmax(pred.logits, dim=1)  # Softmax 적용하여 확률 얻기
next_sentence_label = torch.argmax(probs, dim=1).item()  # 예측된 라벨 얻기
print('최종 예측 레이블 :', next_sentence_label)

최종 예측 레이블 : 1


# 4. 한국어 모델의 다음 문장 예측 모델과 토크나이저

In [12]:
model = BertForNextSentencePrediction.from_pretrained('klue/bert-base')
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [13]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

pred = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])
probs = torch.nn.functional.softmax(pred.logits, dim=1)  # Softmax 적용하여 확률 얻기
next_sentence_label = torch.argmax(probs, dim=1).item()  # 예측된 라벨 얻기
print('최종 예측 레이블 :', next_sentence_label)

최종 예측 레이블 : 0


In [14]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

pred = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])
probs = torch.nn.functional.softmax(pred.logits, dim=1)  # Softmax 적용하여 확률 얻기
next_sentence_label = torch.argmax(probs, dim=1).item()  # 예측된 라벨 얻기
print('최종 예측 레이블 :', next_sentence_label)

최종 예측 레이블 : 1
